<a href="https://colab.research.google.com/github/AkshatMishra29/my_first_project/blob/main/Innomatics_Hackathon_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset overview and problem statement**

1️⃣  Files Provided
You have three different data files, each in a different format, simulating real-world systems.

📂 File 1: orders.csv (Transactional Data) Click here

📂 File 2: users.json (User Master Data) Click here

📂 File 3: restaurants.sql (Restaurant Master Data) Click here

2️⃣  Step-by-Step: Combining the Datasets
Step 1: Load CSV Data
Step 2: Load JSON Data
Step 3: Load SQL Data
Step 4: Merge the Data
Perform joins using keys:
orders.user_id → users.user_id
orders.restaurant_id → restaurants.restaurant_id
Join Type: Left Join (to retain all orders)

Step 5: Create Final Dataset
The final dataset contains:
Order details
User information
Restaurant information
📁 Output File:
 final_food_delivery_dataset.csv
3️⃣ Final Dataset – What Students Should Understand
Students must analyze:
Order trends over time
User behavior patterns
City-wise and cuisine-wise performance
Membership impact (Gold vs Regular)
Revenue distribution and seasonality
📌 This Final dataset is the only source of truth for all questions listed in this Hackathon.



#(**MODULE-1**)


# **Data-Cleaning Using Pandas and Sqlite**

In [52]:
import pandas as pd
import sqlite3
import json


In [53]:
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")


with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript(sql_script)
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()


In [54]:
df = orders.merge(users, on='user_id', how='left')
df = df.merge(restaurants, on='restaurant_id', how='left', suffixes=('', '_rest'))

In [55]:
df = orders.merge(users, on='user_id', how='left')
df = df.merge(restaurants, on='restaurant_id', how='left', suffixes=('', '_rest'))

In [56]:
df.to_csv('final_food_delivery_dataset.csv', index=False)

In [57]:
df.head(10)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_rest,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
5,6,2933,499,12-07-2023,1497.22,Annapurna Tiffins South Indian,User_2933,Chennai,Regular,Restaurant_499,Mexican,4.9
6,7,970,35,30-05-2023,129.21,Royal Biryani North Indian,User_970,Bangalore,Regular,Restaurant_35,Chinese,3.9
7,8,891,57,07-11-2023,269.19,Spice Mess Punjabi,User_891,Pune,Regular,Restaurant_57,Mexican,4.4
8,9,364,7,05-12-2023,953.30,Ruchi Biryani Punjabi,User_364,Chennai,Gold,Restaurant_7,Italian,3.2
9,10,2972,183,30-12-2023,351.41,Taste of Biryani Non-Veg,User_2972,Bangalore,Regular,Restaurant_183,Chinese,3.7


1)*Which city has the highest total revenue(total_amount) from Gold members?*

In [58]:
q1 = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(f"Q1: {q1}")

Q1: Chennai


2)*Which cuisine has the highest average order value across all orders?*

In [59]:
q2 = df.groupby('cuisine')['total_amount'].mean().idxmax()
print(f"Q2: {q2}")

Q2: Mexican


3) *How many distinct users placed the orders worth more than Rs 1000 in total?*

In [60]:
user_totals = df.groupby('user_id')['total_amount'].sum()
q3_count = (user_totals > 1000).sum()
print(f"Q3: {q3_count}")

Q3: 2544


4)*Which restaurant rating range generated the highest total revenue?*

In [61]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)
q4 = df.groupby('rating_range')['total_amount'].sum().idxmax()
print(f"Q4: {q4}")

Q4: 4.6 – 5.0


/tmp/ipython-input-3870172135.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q4 = df.groupby('rating_range')['total_amount'].sum().idxmax()


5)*Among Gold members,which city has the highest average order value?*



In [62]:
q5 = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
print(f"Q5: {q5}")

Q5: Chennai


6)*Which cuisine has the lowest number of distinct restaurants but significant revenue?*

In [63]:
cuisine_stats = df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).sort_values(by='distinct_restaurants')
print(f"Q6: {cuisine_stats.index[0]}")

Q6: Chinese


7)*What percentage of total orders were placed by Gold members?*

In [64]:
gold_orders = (df['membership'] == 'Gold').sum()
q7 = round((gold_orders / len(df)) * 100)
print(f"Q7: {q7}%")

Q7: 50%


8)*Which restaurant has highest average order value but < 20 total orders?*


In [65]:
rest_stats = df.groupby('restaurant_name').agg(
    avg_order_value-=('total_amount', 'mean'),
    order_count=('order_id', 'count')
)
candidates = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
q8 = rest_stats[rest_stats.index.isin(candidates) & (rest_stats['order_count'] < 20)]['avg_order_value'].idxmax()
print(f"Q8: {q8}")-

Q8: Ruchi Foods Chinese


9)*Which combination contibutes the highest revenue?*

In [67]:
combos = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
revenue_map = df.groupby(['membership', 'cuisine'])['total_amount'].sum()
q9 = max(combos, key=lambda x: revenue_map.get(x, 0))
print(f"Q9: {q9[0]} + {q9[1]} cuisine")

Q9: Gold + Italian cuisine


10)*During which quarter of the year is the total revenue highest?*

In [72]:
# Q10: During which quarter of the year is the total revenue highest?
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')
df['quarter'] = df['order_date'].dt.quarter
q10 = df.groupby('quarter')['total_amount'].sum().idxmax()
print(f"Q10: Q{q10}")

Q10: Q3


# **Numerical Questions**

1)**How many total orders were placed by users with Gold membership?**

2)**What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?**

3)**How many distinct users placed at least one order?**

4)**What is the average order value (rounded to 2 decimals) for Gold members?**

5)How many orders were placed for restaurants with rating >= 4.5?

6)How many orders were placed in the top revenue city among Gold members only?

#**Results**

In [83]:
# Importing necessary libraries

import pandas as pd
import sqlite3

# 1. Load Datasets
orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript(sql_script)
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

# 2. Merge Datasets
df = orders.merge(users, on='user_id', how='left')
df = df.merge(restaurants, on='restaurant_id', how='left', suffixes=('', '_rest'))


# 3. Calculate Results
# Total orders by Gold members
total_gold_orders = df[df['membership'] == 'Gold'].shape[0]


# Total revenue in Hyderabad (rounded)
hyd_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())


# Distinct users who placed at least one order
distinct_users = df['user_id'].nunique()


# Avg order value for Gold members (rounded to 2 decimals)
avg_gold_val = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)


# Orders for restaurants with rating >= 4.5
high_rating_orders = df[df['rating'] >= 4.5].shape[0]


# Orders in the top revenue city among Gold members only
gold_df = df[df['membership'] == 'Gold']
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
top_city_gold_orders = gold_df[gold_df['city'] == top_gold_city].shape[0]

# Output values
print(f"1. Total Gold Orders: {total_gold_orders}")
print(f"2. Hyderabad Revenue: {hyd_revenue}")
print(f"3. Distinct Users: {distinct_users}")
print(f"4. Avg Gold Order Value: {avg_gold_val}")
print(f"5. Rating >= 4.5 Orders: {high_rating_orders}")
print(f"6. Orders in {top_gold_city} (Top Gold City): {top_city_gold_orders}")

1. Total Gold Orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct Users: 2883
4. Avg Gold Order Value: 797.15
5. Rating >= 4.5 Orders: 3374
6. Orders in Chennai (Top Gold City): 1337


##**Fill in the blanks(Solutions)**

**This section contains questions with numerical answers. Please enter the correct values, including decimals wherever applicable**

- How many total orders were placed by users with Gold membership?
4987
- What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
1889367
- How many distinct users placed at least one order?
2883
- What is the average order value (rounded to 2 decimals) for Gold members?
797.15
- How many orders were placed for restaurants with rating ≥ 4.5?
3374

- How many orders were placed in the top revenue city among Gold members only?
1337



----


##**fill-in-the-blank questions. Write the correct answer based on the given dataset**  
- The column used to join orders.csv and users.json is __________.
user_id

- The dataset containing cuisine and rating information is stored in __________ format.
..sql

- The total number of rows in the final merged dataset is __________.
10,000

- If a user has no matching record in users.json, the merged values will be __________.
NaN(Not a Value)

- The Pandas function used to combine datasets based on a key is __________.
pd.merge()

- The column membership in the final dataset originates from the __________ file.  
users.json

-The join key used to combine orders data with restaurant details is __________.  
restraunt_id

- The column that helps identify the type of food served by a restaurant is __________.  
cuisine


- If a user places multiple orders, their personal details appear __________ times in the final merged dataset.  
multiple

---


